In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import b3d
import sys
import jax
import jax.numpy as jnp
import genjax
import rerun as rr
sys.path.append(str(b3d.get_root_path()))
from tests.sama4d.data_curation import get_loaders_for_curated_unity_scenes
from tests.sama4d.video_to_tracks.keypoint_tracking_task import KeypointTrackingTask

genjax.pretty()

In [118]:
scene_spec = get_loaders_for_curated_unity_scenes()[0]
# For this task, I have found that all the keypoint indices between 206 and 215
# are on a relatively textured object which is moving over time.
task = KeypointTrackingTask(
    (lambda: (
        scene_spec["feature_track_data_loader"]()
        .remove_points_invisible_at_frame0()
        .sparsify_points_to_minimum_2D_distance_at_frame0(5)
        .slice_keypoints(jnp.array([210]))
    )),
    scene_name=scene_spec["scene_name"],
    preprocessing_fn=(lambda x: x),
    n_frames=20
)

In [122]:
rr.init(f'triangle-fitting-03')
rr.connect("127.0.0.1:8812")
rr.send_blueprint(task.get_rr_blueprint())
task.visualize_task()